In [ ]:
"""
서울시 우리마을가게 상권분석서비스 - 행정동별 매출금액 API 호출
- 기간: 2020년 ~ 2025년 (분기별)
- 업종: 편의점 (CS300002)
- 서비스명: VwsmAdstrdSelngW
"""

import requests
import pandas as pd
import time
from datetime import datetime

# ============================================================
# 설정
# ============================================================
API_KEY = "4a5a5a4c7768666f3634687355484f"  # 여기에 발급받은 인증키 입력
SERVICE_NAME = "VwsmAdstrdSelngW"  # 행정동별 매출금액
BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE_NAME}"

# 편의점 업종코드
CONVENIENCE_STORE_CODE = "CS300002"

# 기준년분기코드 생성 (2020년 1분기 ~ 2025년 4분기)
def generate_quarter_codes(start_year=2022, end_year=2025):
    """년분기 코드 생성: 20221, 20222, 20223, 20224, ..."""
    codes = []
    for year in range(start_year, end_year + 1):
        for quarter in range(1, 5):
            codes.append(f"{year}{quarter}")
    return codes

QUARTER_CODES = generate_quarter_codes(2022, 2025)
print(f"수집 대상 분기: {QUARTER_CODES}")


# ============================================================
# API 호출 함수
# ============================================================
def fetch_all_data():
    """
    전체 데이터 수집 후 편의점만 필터링
    (서울시 API는 업종코드 필터링 파라미터를 제공하지 않음)
    """
    all_data = []
    start = 1
    batch_size = 1000
    
    print(f"\n{'='*60}")
    print("서울시 상권분석 API 데이터 수집 시작")
    print(f"{'='*60}")
    
    while True:
        end = start + batch_size - 1
        url = f"{BASE_URL}/{start}/{end}"
        
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            result = response.json()
            
            # 에러 체크
            if SERVICE_NAME not in result:
                if 'RESULT' in result:
                    print(f"API 에러: {result['RESULT']}")
                break
            
            # 결과 코드 확인
            code = result[SERVICE_NAME]['RESULT']['CODE']
            if code != 'INFO-000':
                print(f"API 응답 코드: {code}")
                break
            
            # 데이터 추출
            rows = result[SERVICE_NAME].get('row', [])
            if not rows:
                print("더 이상 데이터가 없습니다.")
                break
            
            all_data.extend(rows)
            total_count = result[SERVICE_NAME]['list_total_count']
            
            print(f"수집 중: {start:,}~{end:,} / 전체 {total_count:,}건 (현재 {len(all_data):,}건)")
            
            # 모든 데이터 수집 완료 체크
            if len(all_data) >= total_count:
                print(f"\n전체 데이터 수집 완료: {len(all_data):,}건")
                break
            
            start += batch_size
            time.sleep(0.5)  # API 부하 방지
            
        except requests.exceptions.RequestException as e:
            print(f"요청 에러 (start={start}): {e}")
            time.sleep(2)
            continue
        except Exception as e:
            print(f"처리 에러: {e}")
            break
    
    return all_data


def filter_convenience_store(data, quarter_codes=None):
    """
    편의점 데이터 필터링
    - 업종코드: CS300002 (편의점)
    - 기간: 2022년~2025년
    """
    df = pd.DataFrame(data)
    
    # 편의점 필터링
    df_conv = df[df['SVC_INDUTY_CD'] == CONVENIENCE_STORE_CODE].copy()
    print(f"\n편의점 데이터: {len(df_conv):,}건")
    
    # 기준년분기코드 필터링 (2020년~2025년)
    if quarter_codes:
        df_conv['STDR_YYQU_CD'] = df_conv['STDR_YYQU_CD'].astype(str)
        df_conv = df_conv[df_conv['STDR_YYQU_CD'].isin(quarter_codes)]
        print(f"2020~2025년 데이터: {len(df_conv):,}건")
    
    return df_conv


# ============================================================
# 컬럼명 한글 변환
# ============================================================
COLUMN_MAPPING = {
    'STDR_YYQU_CD': '기준_년분기_코드',
    'ADSTRD_CD': '행정동_코드',
    'ADSTRD_CD_NM': '행정동_코드_명',
    'SVC_INDUTY_CD': '서비스_업종_코드',
    'SVC_INDUTY_CD_NM': '서비스_업종_코드_명',
    'THSMON_SELNG_AMT': '당월_매출_금액',
    'THSMON_SELNG_CO': '당월_매출_건수',
    'MDWK_SELNG_AMT': '주중_매출_금액',
    'WKEND_SELNG_AMT': '주말_매출_금액',
    'MON_SELNG_AMT': '월요일_매출_금액',
    'TUES_SELNG_AMT': '화요일_매출_금액',
    'WED_SELNG_AMT': '수요일_매출_금액',
    'THUR_SELNG_AMT': '목요일_매출_금액',
    'FRI_SELNG_AMT': '금요일_매출_금액',
    'SAT_SELNG_AMT': '토요일_매출_금액',
    'SUN_SELNG_AMT': '일요일_매출_금액',
    'TMZON_00_06_SELNG_AMT': '시간대_00~06_매출_금액',
    'TMZON_06_11_SELNG_AMT': '시간대_06~11_매출_금액',
    'TMZON_11_14_SELNG_AMT': '시간대_11~14_매출_금액',
    'TMZON_14_17_SELNG_AMT': '시간대_14~17_매출_금액',
    'TMZON_17_21_SELNG_AMT': '시간대_17~21_매출_금액',
    'TMZON_21_24_SELNG_AMT': '시간대_21~24_매출_금액',
    'ML_SELNG_AMT': '남성_매출_금액',
    'FML_SELNG_AMT': '여성_매출_금액',
    'AGRDE_10_SELNG_AMT': '연령대_10_매출_금액',
    'AGRDE_20_SELNG_AMT': '연령대_20_매출_금액',
    'AGRDE_30_SELNG_AMT': '연령대_30_매출_금액',
    'AGRDE_40_SELNG_AMT': '연령대_40_매출_금액',
    'AGRDE_50_SELNG_AMT': '연령대_50_매출_금액',
    'AGRDE_60_ABOVE_SELNG_AMT': '연령대_60_이상_매출_금액',
    'MDWK_SELNG_CO': '주중_매출_건수',
    'WKEND_SELNG_CO': '주말_매출_건수',
    'MON_SELNG_CO': '월요일_매출_건수',
    'TUES_SELNG_CO': '화요일_매출_건수',
    'WED_SELNG_CO': '수요일_매출_건수',
    'THUR_SELNG_CO': '목요일_매출_건수',
    'FRI_SELNG_CO': '금요일_매출_건수',
    'SAT_SELNG_CO': '토요일_매출_건수',
    'SUN_SELNG_CO': '일요일_매출_건수',
    'TMZON_00_06_SELNG_CO': '시간대_00~06_매출_건수',
    'TMZON_06_11_SELNG_CO': '시간대_06~11_매출_건수',
    'TMZON_11_14_SELNG_CO': '시간대_11~14_매출_건수',
    'TMZON_14_17_SELNG_CO': '시간대_14~17_매출_건수',
    'TMZON_17_21_SELNG_CO': '시간대_17~21_매출_건수',
    'TMZON_21_24_SELNG_CO': '시간대_21~24_매출_건수',
    'ML_SELNG_CO': '남성_매출_건수',
    'FML_SELNG_CO': '여성_매출_건수',
    'AGRDE_10_SELNG_CO': '연령대_10_매출_건수',
    'AGRDE_20_SELNG_CO': '연령대_20_매출_건수',
    'AGRDE_30_SELNG_CO': '연령대_30_매출_건수',
    'AGRDE_40_SELNG_CO': '연령대_40_매출_건수',
    'AGRDE_50_SELNG_CO': '연령대_50_매출_건수',
    'AGRDE_60_ABOVE_SELNG_CO': '연령대_60_이상_매출_건수',
}


# ============================================================
# 메인 실행
# ============================================================
if __name__ == "__main__":
    # 1. API 키 확인
    if API_KEY == "YOUR_API_KEY_HERE":
        print("⚠️  API_KEY를 설정해주세요!")
        print("   서울 열린데이터광장에서 인증키 발급: https://data.seoul.go.kr")
        print("\n[테스트 모드] sample 키로 5건만 조회합니다.")
        API_KEY = "sample"
        BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE_NAME}"
    
    # 2. 데이터 수집
    raw_data = fetch_all_data()
    
    if raw_data:
        # 3. 편의점 + 2022~2025년 필터링
        df_result = filter_convenience_store(raw_data, QUARTER_CODES)
        
        if len(df_result) > 0:
            # 4. 컬럼명 한글 변환
            df_result = df_result.rename(columns=COLUMN_MAPPING)
            
            # 5. 데이터 타입 변환 (금액/건수 숫자로)
            numeric_cols = [col for col in df_result.columns if '금액' in col or '건수' in col]
            for col in numeric_cols:
                df_result[col] = pd.to_numeric(df_result[col], errors='coerce')
            
            # 6. 저장
            output_file = "서울시_편의점_매출_2022_2025.csv"
            df_result.to_csv(output_file, index=False, encoding='utf-8-sig')
            print(f"\n✅ 저장 완료: {output_file}")
            
            # 7. 결과 요약
            print(f"\n{'='*60}")
            print("수집 결과 요약")
            print(f"{'='*60}")
            print(f"총 데이터 수: {len(df_result):,}건")
            print(f"행정동 수: {df_result['행정동_코드_명'].nunique()}개")
            print(f"분기 수: {df_result['기준_년분기_코드'].nunique()}개")
            print(f"\n분기별 데이터 건수:")
            print(df_result.groupby('기준_년분기_코드').size().sort_index())
        else:
            print("편의점 데이터가 없습니다.")
    else:
        print("데이터 수집 실패")